# *btest* initial steps

step-by-step

In [1]:
import argparse
import csv
import itertools
import logging
import os
import sys
import shutil
import time
import math
import random
from scipy.stats import pearsonr
import numpy as np
import pandas as pd
import math
from scipy.stats import spearmanr
from btest import config
from btest import utils
from btest import blockplot
from scipy.stats import t

In [2]:
method = 'spearman'
min_var = 0.0
fdr =0.1

In [3]:
dataX , dataY, featuresX, featuresY  = utils.readData('../demo/X_dataset.txt', '../demo/Y_dataset.txt', min_var=min_var)

Dataset X dimension  after cleaning:  (100, 50)
Dataset Y dimension  after cleaning:  (100, 50)


In [4]:
%%time
within_X = utils.btest_corr(dataX, featuresX, method=method, fdr=fdr, Type='withinX')
within_Y  = utils.btest_corr(dataY, featuresY, method=method, fdr=fdr, Type='withinY')
dataAll = np.concatenate((dataX, dataY), axis=0)
X_Y = utils.btest_corr(dataAll, featuresX, featuresY, method=method, fdr=fdr, Type='X_Y')

CPU times: user 7.61 s, sys: 63.3 ms, total: 7.67 s
Wall time: 7.89 s


In [5]:
X_Y.head()

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold,Type
165,X1,Y66,0.021416,0.327972,49,212.013466,0.65493,X_Y
174,X1,Y75,0.033062,0.305052,49,163.658486,0.65493,X_Y
2747,X27,Y74,0.036443,0.296654,50,120.262541,0.65493,X_Y
173,X1,Y74,0.040235,0.294122,49,99.582635,0.65493,X_Y
1526,X15,Y41,0.042550,0.287992,50,84.249800,0.65493,X_Y


In [6]:
X_Y[(X_Y['Feature_1']=='X0')&(X_Y['Feature_2']=='Y66')]

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold,Type
66,X0,Y66,0.119186,0.223218,50,5.021027,0.65493,X_Y


In [7]:
%%time
X_Y_3 = utils.btest_corr_3(dataAll, featuresX, featuresY, method=method, fdr=fdr, Type='X_Y')

correlation time:  0.05562281608581543
obs count time:  0.0037539005279541016
long transform time:  0.013448953628540039
p-value time:  0.004818916320800781
bh time:  0.0056650638580322266
names time:  0.02114105224609375
sort time:  0.005999088287353516
CPU times: user 108 ms, sys: 4 ms, total: 112 ms
Wall time: 113 ms


In [ ]:
X_Y_3.head()

In [ ]:
X_Y_3[(X_Y_3['Feature_1']=='X0')&(X_Y_3['Feature_2']=='Y66')]

In [ ]:
X_Y_3.loc[:,'Type'].value_counts()

In [ ]:
X_Y[(X_Y['Feature_1']=='X0')&(X_Y['Feature_2']=='Y66')]

In [ ]:
t1 = time.time()
for i in range(1000):
    i
t2 = time.time()
print(t2-t1)

In [13]:
nrow = 5000
ncol = 100
aa = np.array(np.random.random(size=nrow*ncol), dtype=np.float32)
aa[np.random.randint(0, nrow*ncol, size = int(nrow*ncol/20))]=np.nan
aa = aa.reshape((nrow,ncol))
feature_names = ['P'+str(i) for i in range(nrow)]

In [14]:
%%time
test_withinX = utils.btest_corr_3(aa, features=feature_names, method=method, fdr=fdr, Type='withinX')

correlation time:  478.3922049999237
obs count time:  1.197300910949707
long transform time:  2.3605260848999023
p-value time:  2.89923095703125
bh time:  4.529063940048218
names time:  0.05546426773071289
sort time:  5.369394063949585
CPU times: user 8min 12s, sys: 3.27 s, total: 8min 15s
Wall time: 8min 15s


In [15]:
%%time
test_withinX = utils.btest_corr_3(aa, features=feature_names, method='pearson', fdr=fdr, Type='withinX')

correlation time:  3.5690929889678955
obs count time:  1.1708710193634033
long transform time:  2.392249822616577
p-value time:  2.8019051551818848
bh time:  4.636229991912842
names time:  0.03536796569824219
sort time:  20.614816904067993
CPU times: user 33.7 s, sys: 3.51 s, total: 37.2 s
Wall time: 35.8 s


In [16]:
test_withinX.head()

,Feature_1,Feature_2,Correlation,complete_obs,t_statistic,pval,P_adjusted,bh_fdr_threshold,Type
9012470,P2359,P3450,-0.562520,87.0,-6.272721,1.425335e-08,0.178131,0.876291,withinX
10149744,P2832,P4105,-0.534854,91.0,-5.971757,4.723838e-08,0.295181,0.876291,withinX
8195124,P2066,P2402,0.532171,92.0,5.963147,4.776023e-08,0.198961,0.876291,withinX
1114307,P228,P642,0.528063,91.0,5.866354,7.479566e-08,0.233690,0.876291,withinX
5770945,P1331,P3723,0.518548,93.0,5.785201,1.013234e-07,0.253258,0.876291,withinX


In [10]:
df_test = pd.DataFrame(aa.T)

In [12]:
%%time
cr_test = df_test.corr(method = 'spearman')

CPU times: user 19 s, sys: 40.1 ms, total: 19.1 s
Wall time: 19.2 s


In [ ]:
dataAll = dataX
features = featuresX
features_y = None
Type = 'withinX'
iRow = list(range(0, len(features)))
if Type == 'X_Y':
    iCol = list(range(len(features), len(features)+len(features_y)-1))
    features_y = features + features_y
else:
    features_y = features
    iCol = list(range(len(features)))
tests = []

# creating the complete dataset
dataAll2 = pd.DataFrame(dataAll.T, columns=features_y)
cr = dataAll2.corr(method=method)

# calculating t-statistics, based on the correlations
t_stat = (cr*(dataAll2.shape[0]-2)**.5)/(1-cr**2)**.5

# calculating p-values based on the t-statistics
pv = 2 * (1 - t.cdf(abs(t_stat), df=dataAll2.shape[0]-2))
#rho_long_format = cr.stack().reset_index().rename(columns={'level_0':'Feature_1','level_1':'Feature_2', 0:'Correlation'})
#pval_long_format = pv.stack().reset_index().rename(columns={'level_0':'Feature_1','level_1':'Feature_2', 0:'pval'})
#results = pd.concat([pval_long_format, rho_long_format["Correlation"]], axis=1)


In [ ]:
cr = cr.to_numpy()
#pv = pv.to_numpy()

In [ ]:
for i in iRow:
    for j in iCol:
        if i<=j:
            #print(i, j)
            #print(dataAll[i])
            X = dataAll[i]
            Y = dataAll[j]
            nas = np.logical_or(X != X, Y != Y)
            not_na = sum(~nas)
            #X = Y[~nas]
            #Y = Y[~nas]
            #new_X, new_Y = remove_pairs_with_a_missing(X, Y)
            correlation = cr[i, j]
            pval =  pv[i, j]
            tests.append([features[i], features_y[j], pval, correlation, not_na])
results = pd.DataFrame(tests, columns=['Feature_1','Feature_2','pval', 'Correlation', 'Not_NAs'])

p_adust, p_threshold = bh(results["pval"].values, fdr)

results["P_adusted"] = p_adust
results["bh_fdr_threshold"] = p_threshold
results['Type'] = Type
results = results.sort_values(['pval', 'Correlation'],
                              ascending=[True, False])

In [ ]:
# feature names for the complete dataset
featuresAll = featuresX.copy()
featuresAll.extend(featuresY)

In [ ]:
results

In [ ]:
# creating the complete dataset
dataAll_df = pd.DataFrame(dataAll.T, columns=featuresAll)

In [ ]:
%%time
# correlation
cr = dataAll_df.corr(method='pearson')

# calculaing t-statistics, based on the correlations
t_stat = (cr*(dataAll_df.shape[0]-2)**.5)/(1-cr**2)**.5

# calculating p-values based on the t-statistics
pv = 2 * (1 - t.cdf(abs(t_stat), df=dataAll_df.shape[0]-2))

In [ ]:
cr

In [ ]:
%%time
# correlation
cr = dataAll_df.corr(method='pearson')

# calculaing t-statistics, based on the correlations
t_stat = (cr*(dataAll_df.shape[0]-2)**.5)/(1-cr**2)**.5

# calculating p-values based on the t-statistics
pv = 2 * (1 - t.cdf(abs(t_stat), df=dataAll_df.shape[0]-2))

In [ ]:
X_Y

In [ ]:
within_X

In [ ]:
within_Y

In [ ]:
X_Y.pval

In [ ]:
outputpath = "/Users/rah/Documents/omicsEye/btest/demo/btest_output"
os.makedirs(outputpath, exist_ok=True)
X_Y.to_csv(outputpath + '/X_Y.tsv', sep="\t")
#rho_X = rho.iloc[0:500, 0:500]
#rho_X.to_csv(outputpath + '/simtable_X.tsv', sep="\t")
#rho_X_Y.to_csv(outputpath + '/simtable.tsv', sep="\t")
within_X.to_csv(outputpath + '/within_X.tsv', sep="\t")
within_Y.to_csv(outputpath + '/within_Y.tsv', sep="\t")

In [ ]:
#associations = blockplot.load_associations(path = outputpath + '/X_Y.tsv')
associations = blockplot.load_associations('/Users/rah/Dropbox/Ali-Docs/Research_docs/Projects/VastBiome/analysis/btest_bgc_mgx_replaceNA_FALSE_cov_25_measure_rpkg_mbx_pair_spearman_new/X_Y.tsv')

In [ ]:
outputpath

In [ ]:
associations

In [ ]:
#simtable = blockplot.load_order_table(outputpath + '/simtable.tsv', associations)
simtable = blockplot.load_order_table('/Users/rah/Dropbox/Ali-Docs/Research_docs/Projects/VastBiome/analysis/btest_bgc_mgx_replaceNA_FALSE_cov_25_measure_rpkg_mbx_pair_spearman_new//simtable.tsv', associations)

In [ ]:
X_Y

In [ ]:
blockplot.plot(
    simtable,
    associations,
    cmap="RdBu_r",
    mask=False,
    axlabels=["",""],
    outfile="/Users/rah/Documents/omicsEye/btest/demo/btest_output/blockplot.pdf",
    similarity="Spearman"
)